In [1]:
import sys
sys.path

['/usr/src/app/notebooks',
 '/usr/src/app',
 '/usr/src/app/gee_lib',
 '/usr/src/app/ex_ante_lib',
 '/usr/src/app/forestry_carbon_arr',
 '/usr/src/app/fastapi-gee-service',
 '/usr/local/lib/python310.zip',
 '/usr/local/lib/python3.10',
 '/usr/local/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/site-packages']

In [2]:
import ee

In [3]:
from gee_lib.osi.auth import initialize_gee, check_gee_initialization
# Initialize GEE
authenticate = initialize_gee(project_id='ee-iwansetiawan')

# Verify it's working
if check_gee_initialization():
    print("Ready to use GEE!")

✓ GEE Initialized successfully
  Service Account: earth-engine-land-eligibility@ee-iwansetiawan.iam.gserviceaccount.com
  Project ID: ee-iwansetiawan
  Credentials Path: /usr/src/app/user_id.json


In [4]:
import json
import os

### need improvement on how to manage this config, we also in the config the path to the static file
# maybe we need to not put the path in the config.json, don't know yet
# Move to the parent directory of the current script
parent_dir = os.getcwd()

# Construct the absolute path to the JSON file in the 'input' folder
json_path= os.path.join(parent_dir, '00_input', 'balaban_conf.json')

# Read and load the JSON data from the file
with open(json_path, 'r') as file:
    config = json.load(file)

print('config---> ',config)

config--->  {'I_satellite': 'Sentinel', 'pca_scaling': 1, 'tileScale': 2, 'AOI_path': './00_input/LPHD Belaban Rayak.shp', 'OID_field_name': 'id', 'input_training': './00_input/traning_point_merged_belaban.shp', 'algo_ml_selected': 'gbm', 'date_start_end': ['2024-1-1', '2024-5-31'], 'project_name': 'belaban_rayak', 'super_pixel_size': 3, 'region': 'asia', 'pixel_number': 3, 'year_start_loss': 14, 'tree_cover_forest': 30, 'band_name_image': 'Class', 'cloud_cover_threshold': 60, 'crs_input': 'EPSG:4326', 'IsThermal': False, 'fcd_selected': 21, 'high_forest': 65, 'yrf_forest': 45, 'shrub_grass': 45, 'open_land': 30, 'ndwi_hi_sentinel': 0.05, 'ndwi_hi_landsat': 0.1, 'ndwi_hi_planet': -0.2}


In [5]:
## import modules
import osi
import pandas as pd
import geemap

from osi.utils.main import validate_aoi
# convert the modules for image collection (cloudless masking, compositing, reducer etc)
from osi.image_collection.main import ImageCollection
from osi.spectral_indices.spectral_analysis import SpectralAnalysis
from osi.spectral_indices.utils import normalization_100
from osi.hansen.historical_loss import HansenHistorical
from osi.classifying.assign_zone import AssignClassZone
from osi.legends.utils import convert_to_legend_items
from osi.legends.main import LegendsBuilder
from osi.obia.main import OBIASegmentation
from osi.ml.main import LandcoverML
from osi.fcd.main_fcd import FCDCalc
from osi.pca.pca_gee import PCA
from osi.hansen.historical_loss import HansenHistorical
from osi.classifying.assign_zone import AssignClassZone
from osi.arcpy.utils import safe_get_data_source


AOIt_shp_plot = geemap.shp_to_ee(config['AOI_path'])
crs_input = config['crs_input']
I_satellite = config['I_satellite']
project_name = config['project_name']

start_date = config['date_start_end'][0]
end_date = config['date_start_end'][1]

layer_name_image_mosaick = f'image_mosaick_result_ee_{project_name}'

AOI = AOIt_shp_plot
config['AOI'] = AOI

ndwi_hi = 0.1
if config['I_satellite'] == 'Landsat':
    ndwi_hi = config['ndwi_hi_landsat']
elif I_satellite == 'Sentinel':
    ndwi_hi = config['ndwi_hi_sentinel']
elif I_satellite == 'Planet':
    ndwi_hi = config['ndwi_hi_planet']

### Masking and overlay and area helper Make an image out of the AOI area attribute -> convert featurecollection into raster (image) for overlaying tools
OID = config['OID_field_name']
AOI_img = AOI.filter(ee.Filter.notNull([OID])).reduceToImage(
    properties= [OID],
    reducer= ee.Reducer.first()
)

In [6]:
# now starting to do analysis
# initiate instance class for the image collection and later mosaicking
classInputCollection = ImageCollection(I_satellite=I_satellite,
                                       AOI=AOI, 
                                       date_start_end=config['date_start_end'], 
                                       cloud_cover_threshold = config['cloud_cover_threshold'],
                                       region=config['region'])

In [7]:
# run the method from image collection loaded, cloudless compositing until to image_mosaick
image_mosaick = classInputCollection.image_mosaick()

selecting Sentinel images


In [8]:
# Generate Map IDs
### Defining params visualization and layer name
if config['I_satellite'] == 'Planet':
    # true color {"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}
    # nir veg color {"bands":["red","nir","blue"],"min":0,"max":0.6,"gamma":1.5 }
    image_mosaick, vis_params, layer_name = image_mosaick,{'veg_color':{"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}}, f'{I_satellite} mosaicked - {start_date}-{end_date} VegColor'
else:
    image_mosaick, vis_params, layer_name = image_mosaick,{'veg_color':{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 0.6, 'gamma': 1.5 }}, f'{I_satellite} mosaicked - {start_date}-{end_date}'
   

print("Generating GEE Map IDs...")

map_ids = {}
map_ids[layer_name] = image_mosaick.getMapId(vis_params['veg_color'])

print("✓ Map IDs generated for all layers")

# Display tile URLs
for layer_name, map_id_dict in map_ids.items():
    tile_url = map_id_dict['tile_fetcher'].url_format
    print(f"\n{layer_name.upper()}:")
    print(f"  Tile URL: {tile_url}")



Generating GEE Map IDs...
✓ Map IDs generated for all layers

SENTINEL MOSAICKED - 2024-1-1-2024-5-31:
  Tile URL: https://earthengine.googleapis.com/v1/projects/ee-iwansetiawan/maps/c105e89d63ba8761ee46d0d2b2620ffa-e2f1b62f9f06fd41f1eac104a1279dcf/tiles/{z}/{x}/{y}


In [9]:
map_ids

{'Sentinel mosaicked - 2024-1-1-2024-5-31': {'mapid': 'projects/ee-iwansetiawan/maps/c105e89d63ba8761ee46d0d2b2620ffa-e2f1b62f9f06fd41f1eac104a1279dcf',
  'token': '',
  'tile_fetcher': <ee.data.TileFetcher at 0x7f08a6bdd4e0>,
  'image': <ee.image.Image at 0x7f086c279ea0>}}

In [10]:
# import sys
# sys.path.append('/usr/src/app/fastapi-gee-service')
from gee_integration import process_gee_to_mapstore

# Import the AOI fix function properly
from gee_lib.osi.utils import process_aoi_geometry



layers_data = {
    'project_name': project_name,
}

# Option 1: Simple tile URLs only
map_layers = {
    layer_name: map_ids[layer_name]['tile_fetcher'].url_format}


# Fix 1: Use correct map layers
print(f"\n🔍 Available map layers:")
print(f"   map_layers: {len(map_layers)}")

# Fix 2: Fix AOI coordinates using the existing function
print(f"\n🔧 Fixing AOI coordinates...")
AOI_geom = AOI.geometry()
aoi_info = process_aoi_geometry(AOI_geom)

print(f"✅ Fixed AOI Info:")
print(f"   Bbox: {aoi_info['bbox']}")
print(f"   Center: {aoi_info['center']}")

print("🚀 Processing GEE Analysis to MapStore...")
print(f"   Project: {layers_data['project_name']}")
print(f"   Layers: {len(map_layers)}")
print(f"   AOI: {aoi_info['bbox']}")

# Process the complete integration
result = process_gee_to_mapstore(
    map_layers=map_layers,
    project_name=layers_data['project_name'],
    aoi_info=aoi_info,
    clear_cache_first=False,
    fastapi_url="http://fastapi:8000"  # Internal Docker network
)


🔍 Available map layers:
   map_layers: 1

🔧 Fixing AOI coordinates...
✅ Centroid calculated successfully with error margin 1


INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/config/localConfig.json
INFO:gee_integration:Processing GEE analysis: belaban_rayak
INFO:gee_integration:Using simple layer info for 'Sentinel mosaicked - 2024-1-1-2024-5-31': tile_url only
INFO:gee_integration:Registering with FastAPI: belaban_rayak_20251023_203243
INFO:gee_integration:✅ FastAPI registration successful: MapStore catalog updated successfully
INFO:gee_integration:Creating FastAPI proxy URLs for project: belaban_rayak_20251023_203243
INFO:gee_integration:✅ Created 1 FastAPI proxy URLs
INFO:gee_integration:Updating MapStore WMTS: belaban_rayak_20251023_203243
INFO:gee_integration:🔄 Forcing comprehensive WMTS refresh...
INFO:gee_utils:Starting comprehensive WMTS refresh...
INFO:gee_utils:Clearing old WMTS services...
INFO:gee_utils:Found 0 GEE WMTS services
INFO:gee_utils:No GEE WMTS services to remo

Calculated bbox from coordinates: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
✅ AOI processed successfully:
   - Center: [110.46306597258281, -1.803818629688135]
   - Area: Unknown
   - BBox: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
✅ Fixed AOI Info:
   Bbox: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}
   Center: [110.46306597258281, -1.803818629688135]
🚀 Processing GEE Analysis to MapStore...
   Project: belaban_rayak
   Layers: 1
   AOI: {'minx': 110.42625277199998, 'miny': -1.8286048599999736, 'maxx': 110.49828386500002, 'maxy': -1.7805881470000031}


INFO:gee_utils:Refreshing WMTS capabilities...
INFO:gee_utils:WMTS capabilities refreshed successfully
INFO:gee_utils:Updating MapStore WMTS service...
INFO:gee_utils:Successfully updated MapStore WMTS service: gee_analysis_wmts
INFO:gee_utils:Getting current WMTS layers...
INFO:gee_utils:Comprehensive WMTS refresh completed successfully - Found 1 layers
INFO:gee_integration:✅ MapStore WMTS configuration updated
INFO:gee_integration:   New layers found: 1


In [11]:
# !cat /usr/src/app/mapstore/config/localConfig.json